In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# 乱数シードの固定
import os
import numpy as np
import random
import tensorflow as tf
from keras import backend as K

def reset_seed(seed=0):

    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(seed)
    random.seed(seed)

    session_conf = tf.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    tf.set_random_seed(seed)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)

In [0]:
K.clear_session()

# モデルのインスタンス化
reset_seed(0)

In [0]:
import os
import glob
from PIL import Image
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result

In [0]:
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import time

In [0]:
import shutil
counter = 36
for path in glob.glob('/content/gdrive/My Drive/PBL2/pbl_long_20191118/humei/**/*.png', recursive=True):
    shutil.copy2(path, '/content/gdrive/My Drive/pbl_long_data/ok/'+str(counter).zfill(4)+'.png')
    counter+=1
    print(counter)

In [0]:
# 分類するクラス
classes = ['dakon', 'kizu', 'ok']
nb_classes = len(classes)
#画像の大きさを設定
img_width, img_height = 224, 224

# トレーニング用とバリデーション用の画像格納先（パスは自分で設定してください）
train_data_dir = '/content/gdrive/My Drive/pbl_long_data/'
validation_data_dir = '/content/gdrive/My Drive/pbl_long_data/'
for path in glob.glob(train_data_dir + '/**/*.png', recursive=True):
    expand2square(Image.open(path), (0, 0, 0)).resize((224, 224)).save(path, quality=100)

for path in glob.glob(validation_data_dir + '/**/*.png', recursive=True):
    expand2square(Image.open(path), (0, 0, 0)).resize((224, 224)).save(path, quality=100)

nb_train_samples =10  #トレーニングデータ用の画像数
nb_validation_samples = 10  #バリデーション用の画像数
batch_size = 100  #バッチサイズ

In [0]:
# トレーンング用データを生成するジェネレータ作成
train_datagen = ImageDataGenerator(
  rescale=1.0 / 255,
#  zoom_range=0.2,
  width_shift_range=0.1,
  height_shift_range=0.1,
  rotation_range=10,
#  shear_range=0.1,
  horizontal_flip=True,
  vertical_flip=True,
)

train_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  shuffle=True)


In [0]:
# バリデーション用データを生成するジェネレータ作成
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
  validation_data_dir,
  target_size=(img_width, img_height),
  color_mode='rgb',
  classes=classes,
  class_mode='categorical',
  batch_size=batch_size,
  shuffle=True)

In [0]:
# VGG16のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(img_width, img_height, 3))
vgg_conv = VGG16(weights='imagenet',
                 include_top=False,
                 input_tensor=input_tensor)

# 終盤のConv層3つまでパラメータを固定
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False

# モデルの構築
x = Flatten()(vgg_conv.layers[-1].output)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(516, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(nb_classes, activation='softmax')(x)

model = Model(vgg_conv.inputs, x)

model.summary()

optimizer = optimizers.Adam(lr=1e-4)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
data_dir = '/content/gdrive/My Drive/pbl_long_result/'

modelcheckpoint = ModelCheckpoint(data_dir + 'best_model.hdf5', 
                                  monitor='val_loss', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  save_weights_only=False, 
                                  mode='auto'
                                  )

In [0]:
nb_epoch = 100000  #エポック数

# Fine-tuning
history = model.fit_generator(
    train_generator,
    epochs=nb_epoch,
    validation_data=validation_generator,
    callbacks = [modelcheckpoint],
#    samples_per_epoch=nb_train_samples,
#    nb_val_samples=nb_validation_samples
)

In [0]:
model.save(data_dir + 'model.hdf5')

In [0]:
data_dir = '/content/gdrive/My Drive/pbl_long_result/'

# 学習結果を描写
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#acc, val_accのプロット
plt.plot(history.history["acc"], label="acc", ls="-", marker="o")
plt.plot(history.history["val_acc"], label="val_acc", ls="-", marker="x")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(loc="best")
#Final.pngという名前で、結果を保存
plt.savefig(data_dir + 'model_acc.png')
plt.show()

#acc, val_accのプロット
plt.plot(history.history["loss"], label="loss", ls="-", marker="o")
plt.plot(history.history["val_loss"], label="val_loss", ls="-", marker="x")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(loc="best")
#Final.pngという名前で、結果を保存
plt.savefig(data_dir + 'model_loss.png')
plt.show()

## Grad-CAM

In [0]:
import pandas as pd
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [0]:
def Grad_Cam(input_model, x, layer_name):
    '''
    Args:
       input_model: モデルオブジェクト
       x: arary(正規化済み)
       layer_name: 畳み込み層の名前

    Returns:
       jetcam: 影響の大きい箇所を色付けした画像(array)

    '''

    # 前処理
    preprocessed_input = np.expand_dims(x, axis=0)
    preprocessed_input = preprocessed_input.astype('float32')
    preprocessed_input = preprocessed_input / 255.0

    # 予測クラスの算出
    predictions = input_model.predict(preprocessed_input)
    class_idx = np.argmax(predictions[0])
    class_output = input_model.output[:, class_idx]


    #  勾配を取得

    conv_output = input_model.get_layer(layer_name).output   # layer_nameのレイヤーのアウトプット
    grads = K.gradients(class_output, conv_output)[0]  # gradients(loss, variables) で、variablesのlossに関しての勾配を返す
    gradient_function = K.function([input_model.input], [conv_output, grads])  # model.inputを入力すると、conv_outputとgradsを出力する関数

    output, grads_val = gradient_function([preprocessed_input])
    output, grads_val = output[0], grads_val[0]

    # 重みを平均化して、レイヤーのアウトプットに乗じる
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)


    # 画像化してヒートマップにして合成

    cam = cv2.resize(cam, (224, 224), cv2.INTER_LINEAR) # 画像サイズは200で処理したので
    cam = np.maximum(cam, 0) 
    cam = cam / cam.max()

    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x / 2)   # もとの画像に合成

    return jetcam

In [0]:
data_dir = '/content/gdrive/My Drive/pbl_long_result/'
for class_name in classes:
    counter = 1
    for path in glob.glob(validation_data_dir + class_name + '/**/*.png', recursive=True):
        data = img_to_array(Image.open(path))
        gradCAM_IMG = array_to_img(Grad_Cam(model, data, 'block5_conv3'))
        org_IMG = array_to_img(data)
        pred = classes[np.argmax(model.predict(data.reshape(1, data.shape[0], data.shape[1], data.shape[2])))]
        gradCAM_IMG.save(data_dir + 'act_' + class_name + str(counter).zfill(4) + '_pred_' + pred + '_gradCAM.png', quality=100)
        org_IMG.save(data_dir + 'act_' + class_name + str(counter).zfill(4) + '_pred_' + pred + '_org.png', quality=100)
        counter += 1